In [3]:
%load_ext autoreload
%autoreload 2

import cv2, os
import numpy as np

In [4]:
from sklearn.model_selection import train_test_split

images_path = os.path.join(os.getcwd(), 'dataset/ImageSets')
annotations_path = os.path.join(os.getcwd(), 'dataset/SegmentationClass')

images_name = os.listdir(images_path)
annotations_name = os.listdir(annotations_path)

images_name_train, images_name_test, annotations_name_train, annotations_name_test = train_test_split(images_name, annotations_name, train_size=0.8, random_state=42)

In [14]:
from ThresholdModel import ThresholdModel

model = ThresholdModel(images_path, annotations_path)
model.add_label('DiaChua', [255, 221, 51])
model.add_label('VungVoKhuan', [102, 255, 102])
model.add_label('GiayKhangSinh', [240, 120, 240])

model.fit(images_name_train, annotations_name_train)

DiaChua [57.2335197  61.67441013 66.18749084] [118.90048045 128.74450919 137.78265984]
VungVoKhuan [48.34056936 52.64918932 55.3087704 ] [ 96.11223766 102.12618195 107.56167849]
GiayKhangSinh [ 91.67859192  96.99402845 102.36017202] [220.62185685 226.39482088 233.83463581]


In [24]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from ThresholdModel import iou_measure

accuracy = {
    'name': [],
    'DiaChua': [],
    'VungVoKhuan': [],
    'GiayKhangSinh': []
}

for image_name, annotation_name in zip(images_name_test, annotations_name_test):
    image_test = cv2.imread(os.path.join(images_path, image_name))
    annotation_test = cv2.imread(os.path.join(annotations_path, annotation_name))
    
    mask_DiaChua = model.predict(image_test, 'DiaChua')
    mask_VungVoKhuan = model.predict(image_test, 'VungVoKhuan')
    mask_GiayKhangSinh = model.predict(image_test, 'GiayKhangSinh')
    
    accuracy['name'].append(image_name)
    accuracy['DiaChua'].append(iou_measure(mask_DiaChua, model.get_mask(annotation_test, 'DiaChua')))
    accuracy['VungVoKhuan'].append(iou_measure(mask_VungVoKhuan, model.get_mask(annotation_test, 'VungVoKhuan')))
    accuracy['GiayKhangSinh'].append(iou_measure(mask_GiayKhangSinh, model.get_mask(annotation_test, 'GiayKhangSinh')))

    print('%s %.4f %.4f %.4f' % (accuracy['name'][-1], accuracy['DiaChua'][-1], accuracy['VungVoKhuan'][-1], accuracy['GiayKhangSinh'][-1]))

output = pd.DataFrame(accuracy)
output.to_csv('output.csv')

879_IMG_20180911_195128.jpg 0.3425 0.2056 0.0041
873_IMG_20180911_194441.jpg 0.4438 0.6474 0.1110
870_IMG_20180911_194225.jpg 0.5911 0.5175 0.1027
880_IMG_20180911_195205.jpg 0.3418 0.4132 0.0350
868_IMG_20180911_194036.jpg 0.2485 0.7883 0.0583
nadia_box5_75c28c60-77e5-4236-9a1a-828f506caf17.jpg 0.7121 0.2052 0.0460
877_IMG_20180911_194847.jpg 0.3493 0.4473 0.0209
hadeel_box6_3620256a-d309-44ad-9dd7-f649ecf9a82b.jpg 0.1338 0.3473 0.0097
882_IMG_20180911_195412.jpg 0.4991 0.6545 0.0855


In [25]:
output

,name,DiaChua,VungVoKhuan,GiayKhangSinh
0,879_IMG_20180911_195128.jpg,0.342546,0.205640,0.004145
1,873_IMG_20180911_194441.jpg,0.443808,0.647355,0.111040
2,870_IMG_20180911_194225.jpg,0.591059,0.517454,0.102696
3,880_IMG_20180911_195205.jpg,0.341821,0.413190,0.034984
4,868_IMG_20180911_194036.jpg,0.248475,0.788308,0.058259
5,nadia_box5_75c28c60-77e5-4236-9a1a-828f506caf1...,0.712071,0.205207,0.046006
6,877_IMG_20180911_194847.jpg,0.349340,0.447334,0.020887
7,hadeel_box6_3620256a-d309-44ad-9dd7-f649ecf9a8...,0.133761,0.347276,0.009725
8,882_IMG_20180911_195412.jpg,0.499081,0.654529,0.085512
